### Extract Enron Dict Features

In [ ]:
# email dict of sample with all sent Enron emails already created (see enron_corpus_download.py)

#### OLIVIA CHANGE TO OUR FINAL DATA DIRECTORY#####

import os
from email.parser import Parser
import json

path = "/Users/sarahpursley/Documents"
os.chdir(path)

with open('enron_sample.json', 'r') as f:
    email_dict = json.load(f)

email_dict

In [ ]:
# pull the day of the week the email was sent out of enron data
def get_days(dates):
    day_dict = {}
    
    for d in dates:
        d = d.replace(",", "").split(" ")
        day_of_week = d[0]
        if day_of_week in day_dict:
            day_dict[day_of_week] += 1
        else:
            day_dict[day_of_week] = 1
    
    return day_dict

# get all the domains by stripping the to and from email fields and only keeping after the @ sign
def get_domains(emails):
    all_domains = []  

    for email in emails:
        if len(email.split()) > 1:
            continue
        else:
            split = email.split("@")
            domain = split[1]
        
            if domain not in all_domains:
                all_domains.append(domain)

    return all_domains 

In [ ]:
# get Enron dates from email_dict['Date'] values
dates = email_dict['Date']
day_week = get_days(dates)    # use function to get dictioary of day of weeks

df_day = pd.DataFrame.from_dict(day_week, orient= 'index', columns = ['Freq'])

# sort freq in desc order to get most frequent to least frequent
df_result = df_day.sort_values(['Freq'], ascending = False)
# assign score from 0-6 to each day of week
score = range(0, len(df_day['Freq']))

df_result['Score'] = score
df_result

In [ ]:
# check if in valid_domains 
# proxy for a theoretical total email database - check if sender is legitimate
domain_comb = email_dict['To'] + email_dict['From']
valid_domains = get_domains(domain_comb)
print(f"Valid domains extracted: {len(valid_domains)}")

In [ ]:
# get content from dict and make copy
content = email_dict['content']
content2 = content.copy()

# find lengths off all emails
content_char = []
for con in content2:
    content_len = len(con.split())
    if content_len != 0:
        content_char.append(content_len)

# find only the unique lengths 
content_char = list(set(content_char))
print(f"Length of unique email lengths: {len(content_char)}")

In [ ]:
# USE SAME CAPITALIZATION RATIO FROM STYLOMETRICS
# counts number of capital letters and periods in a string and returns the ratio

def count_period_ratio(string):
    """Input is a string of article"""
   # print(string)
    period_cnt = 0
    capital_cnt = 0
    
    for letter in string:
        if letter == ".":
            period_cnt+=1
        elif letter.isupper() == True:
            capital_cnt += 1
    
    ratio=capital_cnt/period_cnt
    return ratio

content = email_dict['content']
content_dict = {}
i = 0

for c in content:
    content_dict[i] = [c, len(c.strip().lstrip())]
    i += 1

for key,value in content_dict.items():
    try:
        ratio = count_period_ratio(value[0].strip().lstrip())
        content_dict[key].append(ratio)
    except ZeroDivisionError:
        content_dict[key].append(0)
        continue


In [ ]:
ratearray = content_dict.values()
# create dataframe from enron email content, email content length, capitilization ratio, and normalized capitilization ratio
df1= pd.DataFrame(ratearray)
df1.columns=["enron_text", "content_len", "ratio"]
df1['norm_ratio'] = df1['ratio']/df1['content_len']
df1.describe()

In [ ]:
import numpy as np
# get average email length 
enron_ratios = df1.groupby(by=["content_len"]).mean().to_dict()
enron_avg_length = round(np.mean(content_char))

# find closest email length in eron dataset
def closest_val(num):
    func = lambda content_char : abs(content_char - num)
    closest = min(content_char, key = func)
    if closest == None:
        return 0
    else:
        return closest

# find if the exact email length exists in the eron dataset
def same_val(num):
    num1 = closest_val(num)
    num2 = num1 in content_char
    
    if num2 == True:
        return "enron email exists with this length"
    else:
        return "enron email does not exist with this length"

# given the fraud length, use the enron cap ratios to pull the capitalization ratio for that enron file 
# for comparison
def match_val(num):
    normalized = enron_ratios['norm_ratio']
    for key, val in normalized.items():
        if key != None and num != None:
            if key == num:
                return val 
            else:
                return 0
            
# match the day of the week for fraud time value        
def match_dates(fraud_date_format):
    
    day_match = {
        'Sun': 'Sunday',
        'Mon': 'Monday',
        'Tue': 'Tuesday',
        'Wed': 'Wednesday',
        'Thu': 'Thursday',
        'Fri': 'Friday',
        'Sat': 'Saturday'}
    
    fraud_date = fraud_date_format.rpartition('T')[0]
    day_of_week = datetime.date(year,month,day)
    full_day = day_of_week.strftime("%A")
    
    #matched_day
    for key,value in day_match.items():
        if value == full_day:
            return key 

# lookup in any domain from fraud data matches a domain in enron dataset
def domain_lookup(domain_list,domain):
    for vd in domain_list:
        if domain == vd:
            return 'found'
        else:
            return 'not found'

In [ ]:
# pull in final scores and valid domains
score_list = df_result.to_dict()['Score']
score_list[None] = None 
valid_domains = get_domains(domain_comb)

# functions to match day of weeks together
def match_dates(fraud_date_format):
    
    day_match = {
        'Sun': 'Sunday',
        'Mon': 'Monday',
        'Tue': 'Tuesday',
        'Wed': 'Wednesday',
        'Thu': 'Thursday',
        'Fri': 'Friday',
        'Sat': 'Saturday'}
    
    fraud_date = fraud_date_format.rpartition('T')[0]
    day_of_week = datetime.date(year,month,day)
    full_day = day_of_week.strftime("%A")
    
    #matched_day
    for key,value in day_match.items():
        if value == full_day:
            return key 

# function to lookup an Enron domain 
def domain_lookup(domain_list,domain):
    for vd in domain_list:
        if domain == vd:
            return 'found'
        else:
            return 'not found'   

### Pull in fraud messages
#### Insert new features into final JSON

In [ ]:
# get fraud dataset
### OLIVE CHANGE THIS TO THE PATH ####
path_file = ''
f = open(path_file) 
messages = json.load(f) 
f.close()


for message in messages:
    content = message["X-TIKA:content"].lstrip().strip()
    closest_enron_value = closest_val(len(content.split()))
    closest_enron_ratio = match_val(closest_enron_value)
    length_exists = same_val(closest_enron_value)
    
    try:
        author_email = message['Message:From-Email']
        domain = author_email.split('@')[-1]
        domain_match = domain_lookup(valid_domains, domain)
    except KeyError:
        author_email = None

    
    try:
        date_created = match_dates(message['Creation-Date'])
        score_created = score_list[date_created]
    except KeyError:
        date_created = None
        score_created = None
        
    try: 
        date_sent = message['MboxParser-from'].strip("  ").split(" ")[2]
        score_sent = score_list[date_sent]
    except KeyError:
        date_sent = None
        score_sent = None
        
    message['fraud msg characters'] = len(content.split())
    message['enron avg. msg characters'] = int(enron_avg_length)
    message['enron find similar characters'] = length_exists
    message['enron closest capitilization ratio'] = closest_enron_ratio
    message['enron capitilzation ratio difference'] = message['capitalization_ratio'] - closest_enron_ratio
    message['enron domain lookup'] = (domain, domain_match)
    message['enron day created score'] = (date_created, score_created)
    message['enron day sent score'] = (date_sent, score_sent)